In [1]:
import os
import re
from typing import List
import joblib
import logging
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, MapType, DoubleType
from pyspark.sql.functions import col

from object_types import CAREER_FRAME, DRAFT, WIKI_PLAYER


spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Parse data") \
    .config("spark.driver.host", "0.0.0.0") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.sql.execution.pythonUTF8StringEncoding", "true") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 09:13:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
WIKI_PLAYER_SCHEMA = StructType([
    StructField("title", StringType(), nullable=False),
    StructField("full_name", StringType(), nullable=False),
    StructField("birthplace", StringType(), nullable=True),
    StructField("career_start", DoubleType(), nullable=True),
    StructField("career_end", DoubleType(), nullable=True),
    StructField("draft", StringType(), nullable=True),
    StructField("draft_year", StringType(), nullable=True),
    StructField("draft_team", StringType(), nullable=True),
    StructField("current_league", StringType(), nullable=True),
    StructField("national_team", StringType(), nullable=True),
    StructField("current_team", StringType(), nullable=True),
    StructField("nationality", StringType(), nullable=True),
    StructField("date_of_birth", StringType(), nullable=True),
    StructField("sex", StringType(), nullable=True)
])

PLAYER_DATA_SCHEMA = StructType([
    StructField("file_path", StringType(), nullable=False),
    StructField("download_url", StringType(), nullable=False),
    StructField("player_name", StringType(), nullable=False),
    StructField("dob", StringType(), nullable=False),
    StructField("position", StringType(), nullable=False),
    StructField("draft_team", StringType(), nullable=True),
    StructField("hand", StringType(), nullable=False),
    StructField("height", DoubleType(), nullable=True),
    StructField("weight", DoubleType(), nullable=True),
    StructField("games_played", DoubleType(), nullable=True),
    StructField("wins", DoubleType(), nullable=True),
    StructField("losses", DoubleType(), nullable=True),
    StructField("ties_ot_losses", DoubleType(), nullable=True),
    StructField("minutes", DoubleType(), nullable=True),
    StructField("shootouts", FloatType(), nullable=True),
    StructField("gaa", FloatType(), nullable=True),
    StructField("save_percentage", FloatType(), nullable=True),
    StructField("goals", DoubleType(), nullable=True),
    StructField("assists", DoubleType(), nullable=True),
    StructField("points", DoubleType(), nullable=True),
    StructField("plus_minus", DoubleType(), nullable=True),
    StructField("point_shares", FloatType(), nullable=True),
    StructField("penalty_minutes", DoubleType(), nullable=True),
    StructField("shots_on_goal", DoubleType(), nullable=True),
    StructField("game_winning_goals", DoubleType(), nullable=True),
])


def load_from_tsv(file, schema) -> DataFrame:
    df = spark.read \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .option("nullValue", None) \
    .option("emptyValue", None) \
    .schema(schema) \
    .csv(os.path.join('.', 'df', file))
    return df 


html_df = load_from_tsv('html_df.tsv', PLAYER_DATA_SCHEMA)
wiki_df = load_from_tsv('wiki_df.tsv', WIKI_PLAYER_SCHEMA)


In [3]:
wiki_df.drop('draft_team')
wiki_df = wiki_df.dropDuplicates(['full_name'])
html_df = html_df.dropDuplicates(['player_name'])

merged_df = html_df.join(wiki_df, (wiki_df['full_name'] == html_df['player_name']) | (html_df['player_name'] == wiki_df['title']), how="left")

In [7]:
merged_df.coalesce(1).write.mode('overwrite') \
        .option('header', 'true') \
        .option('delimiter', '\t') \
        .csv('./folder/')

25/11/23 23:01:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [5]:
wiki_df.count()

19435

In [6]:
html_df.count()

12012

In [7]:
merged_df.count()

12014

In [4]:
html_view = "html"
html_df.createOrReplaceTempView(html_view)

wiki_view = "wiki"
wiki_df.createOrReplaceTempView(wiki_view)

merged_view = "merged"
merged_df.createOrReplaceTempView(merged_view)

In [6]:
spark.sql(f"""
    select goals
    from {merged_view}
    where player_name like '%Crosby'
""").show(truncate=False)

+-----+
|goals|
+-----+
|628.0|
+-----+



In [67]:
spark.sql(f"""
    select *
    from {html_view}
    where player_name = 'Sidney Patrick Crosby'
""").show(truncate=False)

[Stage 112:>                                                        (0 + 1) / 1]

+--------------------------------+---------------------------------------------------------+---------------------+--------------+--------+----------+----+------+------+------------+----+------+--------------+-------+---------+---+---------------+-----+-------+------+----------+------------+---------------+-------------+------------------+
|file_path                       |download_url                                             |player_name          |dob           |position|draft_team|hand|height|weight|games_played|wins|losses|ties_ot_losses|minutes|shootouts|gaa|save_percentage|goals|assists|points|plus_minus|point_shares|penalty_minutes|shots_on_goal|game_winning_goals|
+--------------------------------+---------------------------------------------------------+---------------------+--------------+--------+----------+----+------+------+------------+----+------+--------------+-------+---------+---+---------------+-----+-------+------+----------+------------+---------------+-----------

In [53]:
wiki_df.alias("w1").join(
    wiki_df.alias("w2"),
    (col("w1.full_name") == col("w2.title")) & (col("w1.full_name") != col("w1.title")),
    how="inner"
).select("w1.full_name").distinct().count()

5

In [54]:
merged_df.coalesce(1).write.mode('overwrite') \
        .option('header', 'true') \
        .option('delimiter', '\t') \
        .csv('./merged_dataframe/')

In [35]:
spark.sql(f"""
    select sum(count_name)
    from (
        select count(*) as count_name, full_name
        from {wiki_view}
        group by full_name
        having count_name > 1
    )
""").show()

+---------------+
|sum(count_name)|
+---------------+
|            392|
+---------------+



In [20]:
spark.sql(f"""
    select count(*)
    from {wiki_view}
    where sex = 'f'
""").show()

+--------+
|count(1)|
+--------+
|    1095|
+--------+



In [30]:
spark.sql(f"""
    select count(*)
    from {merged_view}
    where full_name is not null
    
""").show()

[Stage 24:>                                                         (0 + 1) / 1]

+--------+
|count(1)|
+--------+
|    6630|
+--------+

